<center>
<img src="https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/img/this_is_data_anal.png" width="200" height="200"><br>
</center>

# ch02. 텍스트 마이닝 첫걸음



## 2.1 웹 크롤링으로 기초 데이터 수집하기


이번 절에서는 '나무위키 최근 변경 페이지'의 텍스트 데이터를 웹 크롤링으로 수집한 다음, 데이터 내에서 등장한 키워드의 출현 빈도를 분석해 보자.

### - 웹 크롤링 라이브러리 사용하기


파이썬에서는 BeautifulSoup과 requests라는 라이브러리 웹 크롤러를 만들 수 있다. requests는 특정 URL로부터 HTML 문서를 가져오는 작업을 수행하고, BeautifulSoup 모듈은 HTML 문서에서 데이터를 추출하는 작업을 수행한다.

- 페이지의 URL 정보 추출하기

In [33]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [34]:
import requests
from bs4 import BeautifulSoup
import re

# 크롤링할 사이트 주소를 정의한다.
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

In [35]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성한다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)

# 중복 url을 제거한다.
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%98%81
https://namu.wiki/w/%EB%A0%88%EB%93%9C%20%EB%8D%B0%EB%93%9C%20%EB%A6%AC%EB%8E%80%EC%85%98%202/%EB%93%B1%EC%9E%A5%EC%9D%B8%EB%AC%BC
https://namu.wiki/w/%EC%98%A4%EC%A1%B0%EC%98%A4%EC%96%B5
https://namu.wiki/w/%EC%8A%A4%ED%94%84%EB%A6%AC%EA%B1%B4%EC%A6%88
https://namu.wiki/w/%ED%8F%B4%EC%95%84%EC%9B%83%204/%EB%AA%A8%EB%93%9C/%EA%B2%8C%EC%9E%84%ED%94%8C%EB%A0%88%EC%9D%B4


위 코드는 개발자 도구로 살펴본 HTML 구조에 기반하여 table->tbody->tr->td->a 태그 순의 HTML 계층 구조를 좁혀나가는 과정이다. 이 과정을 통해 목표 태그에 도달했을 때, get(href) 함수로 태그의 속성 정보를 추출한다. get()함수는 해당 태그가 가지고 있는 특정한 속성을 추출한다.

### - 텍스트 정보 수집하기


다음 코드는 최근 변경된 문서 중 한 페이지의 텍스트 정보를 크롤링한 것이다. 이전 단계와 다른 점은 get() 함수 대신 text() 함수를 사용하여 태그의 텍스트 정보만을 추출했다는 것이다.

In [36]:
req = requests.get(page_urls[0])
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="article")
title = contents_table.find_all('h1')[0]
category = contents_table.find_all('ul')[0]
content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
content_corpus_list = []

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus = "".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

김정영 


한국 여배우/ㄱ1972년 출생


 [ 펼치기 · 접기 ]소속 아티스트변희봉박해일김유미김강현 이주빈김정영안신우전세현박지영한서준조혜원변성범 권아름 한이수 김수형백현진조아영소속 아티스트변희봉박해일김유미김강현 이주빈김정영안신우전세현박지영한서준조혜원변성범 권아름 한이수 김수형백현진조아영1. 개요2. 영화3. 드라마4. 연극김정영국적대한민국 출생1972년 7월 27일 (48세)가족배우자 김학선직업영화배우, 연극배우신체166cm소속사에스더블유엠피한국의 배우.2000년 실제 상황2002년 나쁜 남자2003년 마들렌2005년 마파도2005년 여고 괴담 4, 목소리2005년 너는 내 운명2008년 우리 학교 대표2014년 우아한 거짓말2015년 헬머니2015년 손님2016년 걷기왕2015년 풍문으로 들었소[1]2016년 tvn 시그널2016년 kbs 2 동네 변호사 조들호2016년 ocn 뱀파이어 탐정2016년 kbs 2 백희가 돌아 왔다2021년 달이 뜨는 강2008년 늦게 배운 피아노2009년 ring ring ring2010년 기묘 여행2011년 매기의 추억2012년 아워 타운2013년 푸른 배 이야기2014년 슬픈 연극2015년 원 파인 데이


## 2.2 나무위키 최근 변경 페이지 키워드 분석하기


이제 분석에 사용할 데이터가 준비 되었으니, 본격적으로 텍스트 마이닝을 알아보자.

### step.1 크롤링: 웹 데이터 가져오기


이전 단계와 동일한 방법으로 웹 데이터를 크롤링한다. 단, 이번에는 모든 URL의 데이터를 가져오자.
다음 코드를 실행하여 나무위키에서 최근 변경이 일어난 페이지들의 URL을 page_urls라는 변수에 저장한다.

- 나무위키의 최근 변경 데이터 크롤링하기

In [37]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비한다.
columns = ['title', 'category', 'content_text']
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '카테고리', '본문' 정보를 데이터 프레임으로 만든다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행한다.
    req = requests.get(page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    contents_table = soup.find(name="article")
    title = contents_table.find_all('h1')[0]
    
    # 카테고리 정보가 없는 경우를 확인한다.
    if len(contents_table.find_all('ul')) > 0:
        category = contents_table.find_all('ul')[0]
    else:
        category = None
        
    content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
    content_corpus_list = []
    
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출한다. 만약 없는 경우, 빈 문자열로 대체한다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
    
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출한다. 만약 없는 경우, 빈 문자열로 대체한다.
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
        
    # 페이지 내 카테고리정보에서 “분류”라는 단어와 개행 문자를 제거한 뒤 추출한다. 만약 없는 경우, 빈 문자열로 대체한다.
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
    
    # 모든 정보를 하나의 데이터 프레임에 저장한다.
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

이제 이 주소들에 다시 한번 접근하여 문서의 본문과 제목, 그리고 카테고리에 등장하는 텍스트 데이터를 가져와보자.

In [38]:
df.head(5)

,title,category,content_text
0,김정영,한국 여배우/ㄱ1972년 출생,[ 펼치기 · 접기 ]소속 아티스트변희봉박해일김유미김강현 이주빈김정영안신우전세현박...
1,레드 데드 리뎀션 2/등장인물,레드 데드 리뎀션 2작품별 캐릭터,상위 문서: 레드 데드 리뎀션 2 은(는) 여기로 연결됩니다. 레드 데드 ...
2,오조오억,인터넷 유행어,가입 후 15일이 지나야 편집 가능한 문서입니다.이 문서는이 문단은 토론을 통...
3,스프리건즈,유희왕/OCG/카드군유희왕/OCG/기계족,한국판 명칭스프리건즈일본판 명칭スプリガンズ영어판 명칭Springans주요 소환법엑시...
4,폴아웃 4/모드/게임플레이,폴아웃 4/모드,상위 문서: 폴아웃 4/모드1. 게임 시스템 변경1.1. 기존 시스템 변경1....


위의 실행 결과는 모든 URL의 텍스트 데이터를 가져온 뒤, 이를 데이터 프레임의 형태로 변환한 것이다. 데이터에 등장하는 불필요한 문자인 '\n','분류'는 크롤링 과정에서 replace() 함수로 제거했다.

### step2 추출: 키워드 정보 추출하기


다음은 수집한 텍스트 데이터에서 키워드 정보를 추출하는 단계이다. 이를 위해 텍스트 전처리 작업이 필요하다. 텍스트 전처리는 특수문자나 외국어를 제거하는 등의 과정을 포함한다. 그런데 이는 언어와 상황마다 조금씩 다르다.
예를 들어 스팸메일을 분류하는 텍스트 마이닝의 경우, 특수문자나 외국어가 분석의 중요한 힌트가 될 수 있기 때문에 이를 제거하지 않는 편이다. 반면, 키워드 분석처럼 '단어'를 추출하는 것이 목적이라면 특정 언어의 글자만을 추출하기도 한다.
파이썬에서는 're'라는 모듈을 통해 정규표현식을 사용할 수 있다. 정규표현식이란 특정한 규칙을 가진 문자열의 집합을 표현하는 형식이다.

- 텍스트 데이터 전처리하기

In [39]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거한다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타낸다.
    result = hangul.sub('', text)
    return result

print(text_cleaning(df['content_text'][0]))

  펼치기  접기 소속 아티스트변희봉박해일김유미김강현 이주빈김정영안신우전세현박지영한서준조혜원변성범 권아름 한이수 김수형백현진조아영소속 아티스트변희봉박해일김유미김강현 이주빈김정영안신우전세현박지영한서준조혜원변성범 권아름 한이수 김수형백현진조아영 개요 영화 드라마 연극김정영국적대한민국 출생년 월 일 세가족배우자 김학선직업영화배우 연극배우신체소속사에스더블유엠피한국의 배우년 실제 상황년 나쁜 남자년 마들렌년 마파도년 여고 괴담  목소리년 너는 내 운명년 우리 학교 대표년 우아한 거짓말년 헬머니년 손님년 걷기왕년 풍문으로 들었소년  시그널년   동네 변호사 조들호년  뱀파이어 탐정년   백희가 돌아 왔다년 달이 뜨는 강년 늦게 배운 피아노년   년 기묘 여행년 매기의 추억년 아워 타운년 푸른 배 이야기년 슬픈 연극년 원 파인 데이


모든 데이터에 전처리를 적용하기 위해선 apply() 함수를 사용한다. 다음 코드는 title,category,content_text 3개의 피처에 apply() 함수를 적용한 것이다.

In [40]:
# 각 피처마다 데이터 전처리를 적용한다.
df['title'] = df['title'].apply(lambda x: text_cleaning(x))
df['category'] = df['category'].apply(lambda x: text_cleaning(x))
df['content_text'] = df['content_text'].apply(lambda x: text_cleaning(x))
df.head(5)

,title,category,content_text
0,김정영,한국 여배우ㄱ년 출생,펼치기 접기 소속 아티스트변희봉박해일김유미김강현 이주빈김정영안신우전세현박지영한...
1,레드 데드 리뎀션 등장인물,레드 데드 리뎀션 작품별 캐릭터,상위 문서 레드 데드 리뎀션 은는 여기로 연결됩니다 레드 데드 온라인의 등장...
2,오조오억,인터넷 유행어,가입 후 일이 지나야 편집 가능한 문서입니다이 문서는이 문단은 토론을 통해 표제...
3,스프리건즈,유희왕카드군유희왕기계족,한국판 명칭스프리건즈일본판 명칭영어판 명칭주요 소환법엑시즈 소환속성종족화염기계족 개...
4,폴아웃 모드게임플레이,폴아웃 모드,상위 문서 폴아웃 모드 게임 시스템 변경 기존 시스템 변경 초반 플레이 설정 변...


다음 과정은 키워드를 추출한 뒤, 빈도 분석을 수행하는 과정이다. 여기서 키워드를 추출한다는 것은 무엇을 의미할까? 키워드 추출이란 좁은 의미에서 명사, 혹은 형태소 단위의 문자열을 추출하는 것이다. 이를 수행하기 위해 말뭉치라는 것을 만들어야 한다.
말뭉치는 말 그대로 텍스트 데이터의 뭉텅이를 의미한다. 
다음 코드에서는 텍스트 피처를 tolist()로 추출한 뒤, join() 함수로 말뭉치를 생성했다.

- 말뭉치 만들기

In [41]:
# 각 피처마다 말뭉치를 생성한다.
title_corpus = "".join(df['title'].tolist())
category_corpus = "".join(df['category'].tolist())
content_corpus = "".join(df['content_text'].tolist())
print(title_corpus)

김정영 레드 데드 리뎀션 등장인물 오조오억 스프리건즈 폴아웃 모드게임플레이  이상 저온사례 하스스톤카드 분포 서다미뷰티풀 군바리 틀파블로뮤직     나무위키관리 도움말 공제수  아스날  시즌리그 메이슨 마운트   악마사냥꾼월드 오브 워크래프트파멸 선진국 마르코원피스작중 행적    책장검지   뉴스 티베트 불교 지푸라기라도 잡고 싶은 짐승들 나무위키프로젝트이미지 벡터화 로보캅 필리포 인자기 드래그마 메탈슬러그 어택유닛정규군  킹덤만화애니메이션 나탈리아 보디아노바 바텀리그 오브 레전드 슈퍼스타 민물장어의 꿈 낙인유희왕 사랑하니까 김정현 다이나제논 드래곤네스트스토리메인퀘스트 바둑 기전 훙하오쉬안 바드룩 그리드템빨 검정 고무신주변인물 호라이즌 레전드 씨 셰퍼드 낙동정맥로 메탈슬러그 어택유닛  파일티스토리 아이콘 제주특별자치도지사 신흥덕 롯데캐슬 레이시티 민선 기 기초의회의원 대항해시대 온라인 파일조효진  런닝맨   서울여자유키카 마리우스 라이트도르프 파일수원역환승센터 과학사한국조선시대       란마루 당신의 마음을 구해드리겠습니다 롯데월드 어드벤처 틀무지개 프레스부르크 갤럭시 퀀텀 흰색이미지 로스트아크흑막설 틀역대 수 황제 육손진삼국무쌍   홈 앤드 어웨이 앨리스 몬태규 더글라스 스캇 플로리다 파일파블로뮤직 페라리  의빈 성씨 


이제 각 말뭉치 안에서 등장하는 형태소를 추출하자. 파이썬의 한국어 형태소 추출 라이브러리 중 가장 쉽게 사용할 수 있는 konlpy를 사용한다.

다음 코드는 konlpy의 Okt tagger를 이용하여 형태소를 추출하는 과정이다. 그런데 추출 키워드를 자세히 보면 한 글자 키워드 혹은 실질적인 의미가 없는 키워드(분류,필요 등의 형태소)들을 발견할 수 있다. 이런 키워드는 어떻게 처리해야 할까?

- konlpy를 이용해 키워드 추출하기

In [45]:
from konlpy.tag import Okt
from collections import Counter

# konlpy의 형태소 분석기로 명사 단위의 키워드를 추출합니다.
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.